# Modeling Sequential Data Using Recurrent Neural Networks

# Introducing sequential data

- What makes sequences unique, compared to other types of data, is that elements in a sequence appear in a certain order and are not independent of each other.

- Typical machine learning algorithms
for supervised learning assume that the input is independent and identically distributed (IID) data,
which means that the training examples are mutually independent and have the same underlying distribution.

- For a sensible example of sequences, consider time
series data, where each example point, x(t), belongs to a particular time, t.

Sequence modeling has many fascinating applications, such as `language translation` (for example,
translating text from English to Hindi), `image captioning`, and `text generation`.



### Sequential data versus time series data

- Time series data is a special type of sequential data where each example is associated with a dimension for time.

- In time series data, samples are taken at successive timestamps, and therefore, the time dimension determines the order among the data points. For example, stock prices and voice or speech records are time series data.

- On the other hand, not all sequential data has the time dimension. For example, in text data or DNA
sequences, the examples are ordered, but text or DNA does not qualify as time series data.

**RNNs can also be used for time series data**

### Why RNNs?
- The standard NN models that we have covered so far, such as multilayer perceptrons (`MLPs`) and `CNNs` for image data, assume that the training examples are independent
of each other and thus do not incorporate ordering information. We can say that such models `do not have a memory of previously seen training examples`.
- For instance, the samples are passed through
the feedforward and backpropagation steps, and the weights are updated independently of the order
in which the training examples are processed.

- `RNNs`, by contrast, are designed for modeling sequences and are `capable of remembering past information and processing new events accordingly`, which is a clear advantage when working with sequence data

## The different categories of sequence modeling

If neither the input nor output data represent sequences, then we are dealing with standard data, and we could simply use a multilayer perceptron (or another classification model) to model such data.

However, `if either the input or output is a sequence`, the modeling task likely falls into one of these categories:

1. **Many-to-One**: The `input data is a sequence, but the output is a fixed-size vector or scalar, not a sequence`.
  - For example, in `sentiment analysis`, the input is text-based (for example, a movie review) and the output is a class label (for example, a label denoting whether a reviewer liked the movie).

2. **One-to-Many**: `The input data is in standard format and not a sequence, but the output is a sequence`.
  - An example of this category is `image captioning`—the input is an image and the output is an English phrase summarizing the content of that image.

3. **Many-to-Many**: `Both the input and output arrays are sequences.` This category can be further
divided based on `whether the input and output are synchronized`.
- An example of a `synchronized`
many-to-many modeling task is `video classification`, where each frame in a video is labeled.
- An example of a `delayed` many-to-many modeling task would be `translating one language into another`. For instance, an entire English sentence must be read and processed by a machine before its translation into Hindi is produced.

# RNNs for modeling sequences

## Understanding the RNN looping mechanism

- In a standard feedforward network, information flows from the input to the hidden layer, and then from
the hidden layer to the output layer.

- On the other hand, in an RNN, the hidden layer receives its input from both the input layer of the current time step and the hidden layer from the previous time step.

- The flow of information in adjacent time steps in the hidden layer allows the network to have a memory of past events.

- This flow of information is usually displayed as a loop, also known as a `recurrent edge` in graph notation, which is how this general RNN architecture got its name.

- As we know, each hidden unit in a standard NN receives only one input—the net preactivation associated with the input layer.

- In contrast, each hidden unit in an RNN receives two distinct sets of input—the `preactivation from the input layer` and the `activation of the same hidden layer from the previous time step, t-1`.

- At the first time step, t=0, the hidden units are initialized to zeros or small random values. Then, at a
time step where t>0, the hidden units receive their input from the data point at the current time, x(t),
and the previous values of hidden units at t-1, indicated as $h^{(t–1)}$.

Similarly, in the case of a multilayer RNN,
- layer=1: Here, the hidden layer is represented as $h_1^{(t)}$ and it receives its input from the data point, x(t), and the hidden values in the same layer, but at the previous time step, $h_1^{(t-1)}$.
- layer=2: The second hidden layer, $h_2^{(t)}$, receives its inputs from the outputs of the layer below
at the current time step ($o_1^{(t)}$) and its own hidden values from the previous time step, $h_2^{(t-1)}$

Since, in this case, each recurrent layer must receive a sequence as input, all the recurrent layers except
the last one must return a sequence as output.
The behavior of the last recurrent layer depends on the type of problem.



## Computing Activations in an RNN

The different weight matrices in a single-layer RNN are as follows:

- $W_{xh}$: The weight matrix between the input, `x(t)`, and the hidden layer, `h`
- $W_{hh}$: The weight matrix associated with the recurrent edge
- $W_{ho}$: The weight matrix between the hidden layer and output layer
- $W_h = [W_{xh}; W_{hh}]$

For the hidden layer, the net input **$z_h(t)$** (pre-activation) is computed through a linear
combination. That is, we compute the sum of the multiplications of the weight matrices with the
corresponding vectors and add the bias unit:

$$
z_h(t) = W_{xh} \cdot x(t) + W_{hh} \cdot h(t-1) + b_h
$$

Then, the activations of the hidden units at time step `t` are calculated as follows:

$$
h(t) = \sigma_h(z_h(t)) = \sigma_h(W_{xh} \cdot x(t) + W_{hh} \cdot h(t-1) + b_h)
$$

Here, $( b_h )$ is the bias vector for the hidden units, and $(\sigma_h)$ is the activation function of the hidden layer.

If we use the concatenated weight matrix \( W_h = [W_{xh}; W_{hh}] \), then the formula for computing the hidden units becomes:

$$
h(t) = \sigma_h\left(
\begin{bmatrix}
W_{xh} \\
W_{hh}
\end{bmatrix}
\begin{bmatrix}
x(t) \\
h(t-1)
\end{bmatrix}
+ b_h
\right)
$$

Once the activations of the hidden units at the current time step are computed, the activations of the output units are calculated as:

$$
o(t) = \sigma_o\left( W_{ho} \cdot h(t) + b_o \right)
$$

## Hidden-recurrence vs. output-recurrence

Now we know the recurrent networks in which the hidden layer has the recurrent property. However, there is an alternative model in which the recurrent connection comes from the output
layer.

In this case, the net activations from the output layer at the previous time step, $o^{t–1}$, can be added
in one of two ways:
- To the hidden layer at the current time step, $h^t$ (output-to-hidden recurrence).
- To the output layer at the current time step, $o^t$ (output-to-output recurrence).

The weights associated with the recurrent connection can
be denoted for the hidden-to-hidden recurrence by $W_{hh}$, for the output-to-hidden recurrence by $W_{oh}$,
and for the output-to-output recurrence by $W_{oo}$.

---

Using the torch.nn module, a recurrent layer can be defined via RNN, which is similar to the
hidden-to-hidden recurrence.

In the following code, we will create a recurrent layer from RNN and perform a forward pass on an input sequence of length 3 to compute the output. We will also manually
compute the forward pass and compare the results with those of RNN.

In [2]:
import torch
import torch.nn as nn

torch.manual_seed(1)

rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh shape:', w_xh.shape)
print('W_hh shape:', w_hh.shape)
print('b_xh shape:', b_xh.shape)
print('b_hh shape:', b_hh.shape)

W_xh shape: torch.Size([2, 5])
W_hh shape: torch.Size([2, 2])
b_xh shape: torch.Size([2])
b_hh shape: torch.Size([2])


In [3]:
x_seq = torch.tensor([[1.0]*5, [2.0]*5, [3.0]*5]).float()

## output of the simple RNN:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))

## manually computing the output:
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1, 5))
    print(f'Time step {t} =>')
    print('   Input           :', xt.numpy())

    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
    print('   Hidden          :', ht.detach().numpy())

    if t>0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))

    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.detach().numpy())
    print('   RNN output      :', output[:, t].detach().numpy())
    print()

Time step 0 =>
   Input           : [[1. 1. 1. 1. 1.]]
   Hidden          : [[-0.4701929   0.58639044]]
   Output (manual) : [[-0.3519801   0.52525216]]
   RNN output      : [[-0.3519801   0.52525216]]

Time step 1 =>
   Input           : [[2. 2. 2. 2. 2.]]
   Hidden          : [[-0.88883156  1.2364398 ]]
   Output (manual) : [[-0.68424344  0.76074266]]
   RNN output      : [[-0.68424344  0.76074266]]

Time step 2 =>
   Input           : [[3. 3. 3. 3. 3.]]
   Hidden          : [[-1.3074702  1.8864892]]
   Output (manual) : [[-0.8649416  0.9046636]]
   RNN output      : [[-0.8649416  0.9046636]]



In our manual forward computation, we used the hyperbolic tangent (tanh) activation function since it
is also used in RNN (the default activation).

As we can see from the printed results, the outputs from the
manual forward computations exactly match the output of the RNN layer at each time step.

---

## The challenges of learning long-range interactions
In Backpropagation through time (BPTT) during training RNNs,vanishing and exploding gradient problems arise in computing the gradients of a loss function

In practice, there are at least three solutions to this problem:

1. **`Gradient clipping`** - here, we specify a cut-off or threshold value for the gradients, and we assign this
cut-off value to gradient values that exceed this value.
2. **Truncated backpropagation through time (`TBPTT`)** - it simply limits the number of time steps that the signal can backpropagate after each forward pass.

While both gradient clipping and TBPTT can solve the exploding gradient problem, the truncation limits
the number of steps that the gradient can effectively flow back and properly update the weights.

3. **`LSTM`**, has been more `successful in vanishing and exploding gradient problems` while modeling l`ong-range dependencies`
through the use of `memory cells`.




## Long Short-Term Memory cells
- The building block of an LSTM is a `memory cell`, which essentially represents or `replaces the hidden layer` of standard RNNs.

- In each memory cell, there is a `recurrent edge` that has the desirable weight, `w=1`, as we discussed, to
overcome the vanishing and exploding gradient problems. The values associated with this recurrent
edge are collectively called the `cell state`.

- The cell state from the previous time step, C(t–1), is modified to get the cell state at the current time step, C(t), without being multiplied directly by any weight factor.

- The flow of information in
this memory cell is controlled by several computation units (often called gates).
<br>

In an LSTM cell, there are three different types of gates, which are known as the `forget gate`, the `input gate`, and the `output gate`:

1. The **forget gate ($f_t$)** allows the memory cell to reset the cell state without growing indefinitely. In fact,
the forget gate `decides which information is allowed to go through and which information to suppress`.

2. The **input gate ($i_t$) and candidate value ($\tilde{C_t}$)** are responsible for `updating the cell state`.

3. The **output gate ($o_t$)** decides how to `update the values of hidden units`.

## Other advanced RNN models

LSTMs provide a basic approach for modeling long-range dependencies in sequences. Yet, it is important to note that there are many variations of LSTMs

Also there is a more recent approach, `gated recurrent unit (GRU)`, which was proposed in 2014. GRUs have a `simpler architecture` than LSTMs; therefore, they are `computationally more efficient`, while their performance in some tasks, such as polyphonic music modeling, is comparable to LSTMs.

---